<a href="https://colab.research.google.com/github/dotnetworm/engineeringNotebooks/blob/master/syncronizeExperimentalData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Syncronize Experimental Data

## Problem Statement

A tank filled up with 250 kg pure water completely. A temperature and pressure sensor located in this tank in order to measure pressure and temperature. These sensors measures temperature and pressures at the random frequencies. The operation starts at 13/08/2023 13:23. Time stamps recorded as a duration as seconds. Temperature readings recorded in "TemperatureReadings.csv" file and pressure readings recorded in "PressureReadings.csv" file.

A thermal engineer needs to calculate input power and requires data in 1 Hz frequency to calculate power input in a certain moment.

Your job as a data engineer create an excel file which contains three column;
A - Time Stamp
B - Temperature
C - Pressure

and data in 1Hz sample rate in the document.

## Loading Required Libraries

In [1]:
import pandas as pd
import numpy as np

### Importing the Files and Create Initial Timestamp

This analysis should be time depended. Initial time has given the problem statement. I will manipulate this csv files and timestamp with pandas functionalities.

In [2]:
df_temp = pd.read_csv("TemperatureReadings.csv")
df_pres = pd.read_csv("PressureReadings.csv")
str_time = "2023-08-13 13:23:00"
initial_time = pd.to_datetime(str_time)

## Exploration of Data Files

In [3]:
df_temp.head(10)

,Time,Temperature
0,s,C
1,0,25
2,0.629504351,25.65055332
3,1.095708579,25.67693117
4,1.306416697,25.9555418
5,1.547389531,26.06290369
6,2.2218921,26.79792763
7,2.901229729,27.26937895
8,3.636332918,27.72315909
9,4.453291276,28.02340695


It can be easily seen the first row of the temperature dataframe contains column headers. Second row of the dataframe contains units. Many DAQ tool export CSV data with these kind of additional information. Pandas doesn't like this additional informations and these should be removed. Let's drop first row.

In [4]:
 df_temp = df_temp.drop([0])
 df_temp.head(10)

,Time,Temperature
1,0,25
2,0.629504351,25.65055332
3,1.095708579,25.67693117
4,1.306416697,25.9555418
5,1.547389531,26.06290369
6,2.2218921,26.79792763
7,2.901229729,27.26937895
8,3.636332918,27.72315909
9,4.453291276,28.02340695
10,5.039672669,28.78498274


In [5]:
df_pres = df_pres.drop([0])
df_pres.head(10)

,Time,Pressure
1,0,1.01325
2,0.306265764,1.694817292
3,0.823788503,2.361269097
4,1.019069161,2.67936125
5,1.6207736,2.713966243
6,1.78794827,3.367685158
7,2.700576719,3.833519863
8,2.821296808,4.421581518
9,3.478214459,5.018844396
10,3.488980792,5.763243432


After dropping first raw, let's  deep dive the data.

In [6]:
df_pres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 1 to 165
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Time      165 non-null    object
 1   Pressure  165 non-null    object
dtypes: object(2)
memory usage: 2.7+ KB


In [7]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 1 to 165
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Time         165 non-null    object
 1   Temperature  165 non-null    object
dtypes: object(2)
memory usage: 2.7+ KB


You can see that, two dataframe columns recorded as object. This caused by the multiple data types was in the column. We dropped first row and rest of our data should be numeric.

Furthermore, out "Time" column in the both dataframes are a duration. Not a complete datatime string. We should convert this column to datetime object using timestamp we have already defined in the begining.


*   Convert all columns to numeric Value
*   Change time column to timedelta object
*   Sum this timedelta values with initial time stamp.
*   Set Time column as an index of the dataframe.

Time column should be set as an index of the data frames for interpolation sections in the further steps.



In [8]:
df_pres["Time"] = pd.to_numeric(df_pres["Time"])
df_temp["Time"] = pd.to_numeric(df_temp["Time"])
df_pres["Pressure"] = pd.to_numeric(df_pres["Pressure"])
df_temp["Temperature"] = pd.to_numeric(df_temp["Temperature"])

In [9]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 1 to 165
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time         165 non-null    float64
 1   Temperature  165 non-null    float64
dtypes: float64(2)
memory usage: 2.7 KB


In [10]:
df_pres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 1 to 165
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time      165 non-null    float64
 1   Pressure  165 non-null    float64
dtypes: float64(2)
memory usage: 2.7 KB


In [11]:
df_pres["Time"] = df_pres["Time"].round(2)

In [12]:
df_temp["Time"] = df_temp["Time"].round(2)

In [13]:

df_pres["Time"] = initial_time + pd.to_timedelta(df_pres["Time"], unit="S")
df_temp["Time"] = initial_time + pd.to_timedelta(df_temp["Time"], unit="S")
df_pres = df_pres.set_index("Time")
df_temp = df_temp.set_index("Time")

Let's investigate the dataframes.

In [14]:
df_pres.head()

,Pressure
Time,
2023-08-13 13:23:00.000,1.013250
2023-08-13 13:23:00.310,1.694817
2023-08-13 13:23:00.820,2.361269
2023-08-13 13:23:01.020,2.679361
2023-08-13 13:23:01.620,2.713966


In [15]:
df_pres.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 165 entries, 2023-08-13 13:23:00 to 2023-08-13 13:24:17.040000
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pressure  165 non-null    float64
dtypes: float64(1)
memory usage: 2.6 KB


In [16]:
df_temp.head()

,Temperature
Time,
2023-08-13 13:23:00.000,25.000000
2023-08-13 13:23:00.630,25.650553
2023-08-13 13:23:01.100,25.676931
2023-08-13 13:23:01.310,25.955542
2023-08-13 13:23:01.550,26.062904


In [17]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 165 entries, 2023-08-13 13:23:00 to 2023-08-13 13:24:25.030000
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  165 non-null    float64
dtypes: float64(1)
memory usage: 2.6 KB


In [18]:
df_meas = pd.concat([df_temp, df_pres], axis=1)

In [19]:
df_meas.head(10)

,Temperature,Pressure
Time,,
2023-08-13 13:23:00.000,25.000000,1.013250
2023-08-13 13:23:00.310,NaN,1.694817
2023-08-13 13:23:00.630,25.650553,NaN
2023-08-13 13:23:00.820,NaN,2.361269
2023-08-13 13:23:01.020,NaN,2.679361
2023-08-13 13:23:01.100,25.676931,NaN
2023-08-13 13:23:01.310,25.955542,NaN
2023-08-13 13:23:01.550,26.062904,NaN
2023-08-13 13:23:01.620,NaN,2.713966


In [20]:
indexer = pd.date_range(start=df_meas.index[0], end=df_meas.index[-1], freq="1ms")
df_meas = df_meas.reindex(indexer)
df_meas = df_meas.interpolate(method="time")

In [21]:
df_meas.head(10000)

,Temperature,Pressure
2023-08-13 13:23:00.000,25.000000,1.013250
2023-08-13 13:23:00.001,25.001033,1.015448
2023-08-13 13:23:00.002,25.002065,1.017647
2023-08-13 13:23:00.003,25.003098,1.019846
2023-08-13 13:23:00.004,25.004130,1.022044
...,...,...
2023-08-13 13:23:09.995,31.076329,12.200439
2023-08-13 13:23:09.996,31.076402,12.202266
2023-08-13 13:23:09.997,31.076475,12.204094
2023-08-13 13:23:09.998,31.076548,12.205923


In [22]:
df_meas.resample("S").ffill()

,Temperature,Pressure
2023-08-13 13:23:00,25.000000,1.013250
2023-08-13 13:23:01,25.671319,2.647552
2023-08-13 13:23:02,26.556576,3.475185
2023-08-13 13:23:03,27.330701,4.584471
2023-08-13 13:23:04,27.856603,6.032195
...,...,...
2023-08-13 13:24:21,100.107785,89.209838
2023-08-13 13:24:22,101.274881,89.209838
2023-08-13 13:24:23,101.737653,89.209838
2023-08-13 13:24:24,102.163903,89.209838
